In [14]:
import os
import sys
import torch
import tqdm as tq
import numpy as np
import pandas as pd
import torch.nn as nn
import scipy.sparse as sp
import torch.optim as optim
import torch.utils.data as data
from torch.utils.tensorboard import SummaryWriter
sys.path.append('/home/shape3d/code/RecSys/Recommendation System')

## Dataset preparation

Load data from files

In [15]:
def load_all():
	train_data = pd.read_csv(
		'data/ml-1m.train.rating', 
		sep='\t', header=None, names=['user', 'item'], 
		usecols=[0, 1], dtype={0: np.int32, 1: np.int32})

	user_num = train_data['user'].max() + 1
	item_num = train_data['item'].max() + 1

	train_data = train_data.values.tolist()

	train_mat = sp.dok_matrix((user_num, item_num), dtype=np.float32)
	for x in train_data:
		train_mat[x[0], x[1]] = 1.0

	test_data = []
	with open('data/ml-1m.test.negative', 'r') as fd:
		line = fd.readline()
		while line != None and line != '':
			arr = line.split('\t')
			u = eval(arr[0])[0]
			test_data.append([u, eval(arr[0])[1]])
			for i in arr[1:]:
				test_data.append([u, int(i)])
			line = fd.readline()
	return train_data, test_data, user_num, item_num, train_mat

train_data, test_data, user_num ,item_num, train_mat = load_all()


In [16]:
class DatasetClass(data.Dataset):
    def __init__(self, features, num_item, train_mat = None, num_ng = 0, is_training = None):
        super(DatasetClass, self).__init__()
        self.features_ps = features
        self.num_item = num_item
        self.train_mat = train_mat
        self.num_ng = num_ng
        self.is_training = is_training
        self.labels = [0 for _ in range(len(features))]

    def ng_sample(self):
        self.features_ng = []
        for x in self.features_ps:
            u = x[0]
            for t in range(self.num_ng):
                j = np.random.randint(self.num_item)
                while (u, j) in self.train_mat:
                    j = np.random.randint(self.num_item)
                self.features_ng.append([u, j])

        labels_ps = [1 for _ in range(len(self.features_ps))]
        labels_ng = [0 for _ in range(len(self.features_ng))]

        self.features_fill = self.features_ps + self.features_ng
        self.labels_fill = labels_ps + labels_ng

    def __len__(self):
        return (self.num_ng + 1) * len(self.labels)

    def __getitem__(self, idx):
        features = self.features_fill if self.is_training else self.features_ps
        labels = self.labels_fill if self.is_training else self.labels

        user = features[idx][0]
        item = features[idx][1]
        label = labels[idx]
        return user, item ,label
		

Create dataloader objects

In [17]:
train_dataset = DatasetClass(train_data, item_num, train_mat, 4, True)
test_dataset = DatasetClass(test_data, item_num, train_mat, 0, False)

train_dataloader = data.DataLoader(train_dataset, batch_size = 256, shuffle=True)
test_dataloader = data.DataLoader(test_dataset, batch_size = 100, shuffle=False)

## Define the Model

In [18]:
class NCF(nn.Module):
    def __init__(self, user_num, item_num, factor_num, num_layers = 3):
        super(NCF, self).__init__()

        self.embed_user_GMF = nn.Embedding(user_num, factor_num)
        self.embed_item_GMF = nn.Embedding(item_num, factor_num)
        self.embed_user_MLP = nn.Embedding(
                user_num, factor_num * (2 ** (num_layers - 1)))
        self.embed_item_MLP = nn.Embedding(
                item_num, factor_num * (2 ** (num_layers - 1)))

        MLP_modules = []
        for i in range(num_layers):
            input_size = factor_num * (2 ** (num_layers - i))
            MLP_modules.append(nn.Linear(input_size, input_size//2))
            MLP_modules.append(nn.ReLU())
        self.MLP_layers = nn.Sequential(*MLP_modules)

        predict_size = factor_num * 2
        self.predict_layer = nn.Linear(predict_size, 1)

    def forward(self, user, item):
        embed_user_GMF = self.embed_user_GMF(user)
        embed_item_GMF = self.embed_item_GMF(item)
        output_GMF = embed_user_GMF * embed_item_GMF

        embed_user_MLP = self.embed_user_MLP(user)
        embed_item_MLP = self.embed_item_MLP(item)
        interaction = torch.cat((embed_user_MLP, embed_item_MLP), -1)
        output_MLP = self.MLP_layers(interaction)

        concat = torch.cat((output_GMF, output_MLP), -1)

        prediction = self.predict_layer(concat)
        return prediction.view(-1)


## Instantiate the model, optimizer and loss function

In [19]:
model = NCF(user_num, item_num, factor_num = 16)
model.cuda()

loss_function = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

## Define metrics and validation loop

In [20]:
def hit(gt_item, pred_items):
    if gt_item in pred_items:
        return 1
    return 0

def ndcg(gt_item, pred_items):
	if gt_item in pred_items:
		index = pred_items.index(gt_item)
		return np.reciprocal(np.log2(index+2))
	return 0

def validation(model, test_loader, top_k):
	HR_10, HR_1, NDCG_10, NDCG_1 = [], [], [], []

	for user, item, label in test_loader:
		user = user.cuda()
		item = item.cuda()

		predictions = model(user, item)
		_, indices = torch.topk(predictions, top_k)
		recommends = torch.take(
				item, indices).cpu().numpy().tolist()

		gt_item = item[0].item()
		HR_10.append(hit(gt_item, recommends))
		HR_1.append(hit(gt_item, [recommends[0]]))
		NDCG_10.append(ndcg(gt_item, recommends))
		NDCG_1.append(ndcg(gt_item, [recommends[0]]))

	return np.mean(HR_10), np.mean(HR_1), np.mean(NDCG_10), np.mean(NDCG_1)

## Training

Setup Tensorboard

In [21]:
writer = SummaryWriter('log') # start tensorboard
for f in os.listdir('log'):
    os.remove('log/' + f)

Define the train loop

In [22]:
epochs = 40

def train_loop():
	count, best_ndcg_10 = 0, 0
	for epoch in tq.tqdm(range(epochs)):
		model.train()
		train_dataloader.dataset.ng_sample()

		epoch_loss = 0
		for user, item, label in train_dataloader:
			user = user.cuda()
			item = item.cuda()
			label = label.float().cuda()

			model.zero_grad()
			prediction = model(user, item)
			loss = loss_function(prediction, label)
			loss.backward()
			optimizer.step()
			writer.add_scalar('Iteration loss', loss.item(), count)
			epoch_loss += loss.item()
			count += 1

		model.eval()
		HR_10, HR_1, NDCG_10, NDCG_1 = validation(model, test_dataloader, top_k = 10)
		writer.add_scalar('Epoch loss', epoch_loss/len(train_dataloader), epoch)
		writer.add_scalar('HR@10', HR_10, epoch)
		writer.add_scalar('HR@1', HR_1, epoch)
		writer.add_scalar('NDCG@10', NDCG_10, epoch)
		writer.add_scalar('NDCG@1', NDCG_1, epoch)

		print("HR: {:.3f}\tNDCG@10: {:.3f}\tNDCG@1: {:.3f}".format(np.mean(HR_10), np.mean(NDCG_10), np.mean(NDCG_1)))

		if NDCG_10 > best_ndcg_10:
			best_hr, best_ndcg_10, best_ndcg_1, best_epoch = HR_10, NDCG_10, NDCG_1, epoch
			torch.save(model, 'ckpt/best_NDCG.pth')

	print("End. Best epoch {:03d}: HR@10 = {:.3f}, NDCG@10 = {:.3f} NDCG@1 = {:.3f}".format(best_epoch, best_hr, best_ndcg_10, best_ndcg_1))

train_loop()

  2%|▎         | 1/40 [01:14<48:15, 74.25s/it]

HR: 0.454	NDCG@10: 0.252	NDCG@1: 0.101


  5%|▌         | 2/40 [02:28<46:55, 74.09s/it]

HR: 0.463	NDCG@10: 0.258	NDCG@1: 0.102


  8%|▊         | 3/40 [03:42<45:49, 74.31s/it]

...

HR: 0.680	NDCG@10: 0.410	NDCG@1: 0.187


 98%|█████████▊| 39/40 [47:31<01:08, 68.07s/it]

HR: 0.684	NDCG@10: 0.410	NDCG@1: 0.187


100%|██████████| 40/40 [48:37<00:00, 72.93s/it]

HR: 0.682	NDCG@10: 0.412	NDCG@1: 0.189
End. Best epoch 033: HR@10 = 0.688, NDCG@10 = 0.414 NDCG@1 = 0.188
